<a href="https://colab.research.google.com/github/ragyeongyoon/langchain-playground/blob/main/3_2_TokenTextSplitter_tiktoken%2C_spacy%2C_nltk%2C_konlpy%2C_stransformer%2C_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 필수 패키지 설치
- langchain
- tiktoken
- https://python.langchain.com/docs/concepts/tokens/
- https://python.langchain.com/docs/how_to/split_by_token/#tiktoken
- https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TokenTextSplitter.html

In [ ]:
!pip install -qU langchain
!pip install -qU langchain_community

2. Tiktoken - OpenAI
- https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken

Tiktoken은 OpenAI에서 사용하는 토크나이저이다. 토크나이저란 문장을 토큰이라는 작은 단위로 쪼개는 도구이다.

In [ ]:
!pip install -qU tiktoken

WebBaseLoader
웹 페이지의 내용을 가져오는 Langchain의 도구이다.

bs4.SoupStrainer
웹 페이지의 특정 부분만 선택하여 가져오기 위해 사용한다. 예제에서는 id가 jwxe_main_content인 <section> 태그 안의 내용만 가져옵니다.

document.page_content: 크롤링된 웹 페이지의 텍스트 내용을 담고 있다. len을 통해 크롤링한 텍스트의 전체 길이를 출력한다.

In [ ]:
from langchain.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader('https://ku-sw.korea.ac.kr/swuniv/about/intro.do', bs_kwargs={'parse_only':bs4.SoupStrainer('section', attrs={'id':'jwxe_main_content'})})
document = loader.load()[0]
len(document.page_content)

CharacterTextSplitter: 텍스트를 지정된 chunk_size에 따라 글자 단위로 자릅니다.

* chunk_size: 분할된 텍스트 조각(청크)의 최대 크기를 글자 수로 지정합니다.
* chunk_overlap: 청크 간에 중복될 글자 수를 지정합니다.
* length_function: 텍스트 길이를 계산하는 함수를 지정합니다. 기본값은 len입니다.


text_splitter을 사용하여 document.page_content라는 긴 텍스트를 실제로 자르게 명령합니다. 잘린 조각은 list 형태로 texts 변수에 저장됩니다. 전체 텍스트(len=691)를 5개의 조각으로 잘랐으며, 2번째 조각의 글자 수는 166자입니다.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0, length_function=len)
texts = text_splitter.split_text(document.page_content)
len(texts), len(texts[1]), texts[1]

RecursiveCharacterTextSplitter : 텍스트 자르는 방식이 더 정교한 분할기.
재귀적(recursive), 다양한 구분자를 사용해 텍스트를 자른다.

* (문단 기준, 줄바꿈 기준, 공백 기준, 글자 단위)
* 전체 텍스트(len=691)를 9개의 조각으로 잘랐으며, 2번째 조각의 글자 수는 97자이다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0, length_function=len)
texts = text_splitter.split_text(document.page_content)
len(texts), len(texts[1]), texts[1]

tiktoken: OpenAI의 모델들이 텍스트를 처리하는 방식과 동일하게 토큰 수를 계산하고 텍스트를 분할할 수 있게 해주는 라이브러리

* 토큰(Token): 자연어 처리 모델이 텍스트를 이해하는 가장 기본적인 의미 단위.

* 글자 수와 토큰 수의 관계: 글자 수와 토큰 수는 다르며, 특히 한국어는 영어에 비해 글자 수보다 토큰 수가 훨씬 많다. 영어는 보통 한 단어가 1토큰에 가깝지만, 한국어는 글자 하나가 여러 개의 바이트(byte) 단위로 쪼개져 토큰으로 표현되는 경우가 많기 때문이다.

-> 결과: 97개의 글자로 이루어진 문장이 'davinci' 모델 기준으로는 169개의 토큰으로 변환됨


In [ ]:
import tiktoken

encoder = tiktoken.encoding_for_model('davinci')
print([encoder.decode_single_token_bytes(t).decode(errors='ignore') for t in encoder.encode(texts[-1])])
len(texts[1]), texts[1], len(texts[1]), len(encoder.encode(texts[1]))

RecursiveCharacterTextSplitter가 텍스트를 자르는 기준을 글자 수에서 토큰(Token) 수로 변경

tiktoken_len 함수 정의: text 입력받아 그 text의 tocken 수를 반환

RecursiveCharacterTextSplitter에서, 직접 만든 tiktoken_len 함수를 지정함. chunk_size=100은 이제 '100 글자'가 아닌 '100 토큰'을 의미하게 된다. (분할기에서, 글자가 아닌 토큰 수를 기준으로 텍스트를 자르게 된다)

> 전체 텍스트는 16조각으로 나뉘었고, 두 번째 조각의 글자 수는 50자, 두 번째 조각의 토큰 수는 92개(chunk_size=100 토큰 제한)


In [ ]:
def tiktoken_len(text):
    return len(encoder.encode(text))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0, length_function=tiktoken_len)
texts = text_splitter.split_text(document.page_content)
len(texts), len(texts[1]), tiktoken_len(texts[1]), texts[1]

RecursiveCharacterTextSplitter가 텍스트를 자르는 기준을 davinci 모델용 토크나이저로 변경함

RecursiveCharacterTextSplitter에서, chunk_size가 자동으로 토큰 수를 의미하게 된다.

> 전체 텍스트는 16조각으로 나뉘었고, 두 번째 조각의 글자 수는 50자, 두 번째 조각의 토큰 수는 92개(chunk_size=100 토큰 제한)







In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(model_name='davinci', chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(document.page_content)
len(texts), len(texts[1]), tiktoken_len(texts[1]), texts[1]

---
반복문을 통한 분할 결과 확인: for 반복문을 사용하여 RecursiveCharacterTextSplitter로 잘라놓은 텍스트 조각들의 리스트(texts)에서 TextSplitter가 만들어낸 모든 텍스트 조각(chunk)을 하나씩 검토함. 각 조각의 글자 수, 토큰 수, 실제 내용을 확인할 수 있다.

> 16개 조각 중, 각 조각의  글자 수, 토큰 수, 실제 내용
> 모든 줄에서 토큰 수가 글자 수보다 훨씬 크다(한국어 텍스트의 처리 특징)





In [ ]:
for text in texts:
    print(len(text), len(encoder.encode(text)))
    print(text)

3. SpaCy
- https://spacy.io/usage/linguistic-features#tokenization
- https://spacy.io/api/tokenizer
- https://spacy.io/models/
- https://universaldependencies.org/u/pos/

In [ ]:
!pip install -qU spacy
# !python -m spacy download en_core_web_sm
# !python -m spacy download ko_core_news_sm

In [ ]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = English()
tokenizer = Tokenizer(nlp.vocab)

tokenizer('To be or not to be.'), tokenizer.explain('To be or not to be.'), ','.join([_.text for _ in nlp.vocab if _.is_stop])

In [ ]:
tokenizer = nlp.tokenizer
tokenizer('To be or not to be.'), tokenizer.explain('To be or not to be.')

In [ ]:
nlp = spacy.load('en_core_web_sm')

for token in nlp('To be or not to be.'):
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [ ]:
nlp = spacy.load('ko_core_news_sm')

for token in nlp(document.page_content)[4:15]:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

In [ ]:
len(nlp.tokenizer(document.page_content)), '|'.join([token.text for token in nlp.tokenizer(document.page_content)][4:15])

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], len(texts[1])

In [ ]:
from langchain.text_splitter import SpacyTextSplitter

nlp = spacy.load('en_core_web_sm')

def spacy_len(text):
    return len(nlp(text))

text_splitter = SpacyTextSplitter(pipeline='en_core_web_sm', chunk_size=100, chunk_overlap=0)#, length_function=spacy_len)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], [spacy_len(text) for text in texts], '|'.join([token.text for token in nlp(texts[1])])

In [ ]:
nlp = spacy.load('ko_core_news_sm')

def spacy_len(text):
    return len(nlp(text))

text_splitter = SpacyTextSplitter(pipeline='ko_core_news_sm', chunk_size=100, chunk_overlap=0)#, length_function=spacy_len)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], [spacy_len(text) for text in texts], '|'.join([token.text for token in nlp(texts[1])])

In [ ]:
text_splitter = SpacyTextSplitter(pipeline='ko_core_news_sm', chunk_size=100, chunk_overlap=0, length_function=spacy_len)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], [spacy_len(text) for text in texts], texts[0]

## 4. NLTK
- https://python.langchain.com/api_reference/text_splitters/nltk/langchain_text_splitters.nltk.NLTKTextSplitter.html

In [ ]:
!pip install -qU nltk

In [ ]:
from nltk.downloader import download
download('punkt_tab')

In [ ]:
from langchain.text_splitter import NLTKTextSplitter

def nltk_len(text):
    return len(word_tokenize(text))

text_splitter = NLTKTextSplitter(chunk_size=100, chunk_overlap=0)#, length_function=nltk_len)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], [len(word_tokenize(text)) for text in texts], '|'.join([token for token in word_tokenize(texts[3])])

In [ ]:
from nltk.tokenize import word_tokenize

len(word_tokenize(texts[3]))

In [ ]:
text_splitter = NLTKTextSplitter(chunk_size=100, chunk_overlap=0, length_function=nltk_len)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], [len(word_tokenize(text)) for text in texts], texts[0]

## 5. KoNLPy
- https://python.langchain.com/docs/how_to/split_by_token/#konlpy

In [ ]:
!pip install -qU konlpy

In [ ]:
from konlpy.tag import Kkma

ma = Kkma().morphs
ss = Kkma().sentences

In [ ]:
from langchain.text_splitter import KonlpyTextSplitter

def konlpy_len(text):
    return len(ma(re.sub('\s', '', text)))

text_splitter = KonlpyTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], [len(ma(text)) for text in texts], '|'.join([token for token in ma(texts[1])])

In [ ]:
import re

text_splitter = KonlpyTextSplitter(chunk_size=100, chunk_overlap=0, length_function=konlpy_len)
texts = text_splitter.split_text(re.sub('\s', ' ', document.page_content))
len(texts), [len(text) for text in texts], [len(ma(text)) for text in texts], '|'.join([token for token in ma(texts[1])])

##6. SentenceTransformer
- https://python.langchain.com/docs/how_to/split_by_token/#sentencetransformers
- https://huggingface.co/models?library=sentence-transformers&language=ko&sort=trending

---
**SentenceTransformersTokenTextSplitter**: **Sentence-Transformers** 라이브러리의 임베딩 모델에 포함된 토크나이저를 기준으로 텍스트를 분할하는 도구

(별도의 model_name을 설정하지 않으면, sentence-transformers/all-mpnet-base-v2라는 기본 모델(주로 영어에 최적화됨)의 토크나이저를 사용)

> 전체 텍스트가 3개 조각으로 나눠짐, 각 조각의 글자 수는 583자, 518자, 51자로 큰 조각으로 나옴. chunk_size=100 (토큰)으로 설정했음에도 불구하고 **분할기가 사용하는 기본 토크나이저가 한국어를 효율적으로 처리하지 못하여**, 적은 수의 토큰(100개)이 매우 많은 수의 글자를 포함함

In [ ]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], [len(text) for text in texts]

---
AutoTokenizer: Hugging Face의 transformers 라이브러리에 있는 도구로, 모델 이름만으로 해당 모델에 맞는 토크나이저를 자동으로 불러올 수 있음

> 오류 메세지: 최대 시퀀스 길이 (Maximum Sequence Length): 대부분의 언어 모델은 한 번에 처리할 수 있는 토큰 수에 한계가 있다 (이 모델의 경우 512 토큰). 텍스트 분할(Text Splitting)은 이 한계 때문에 반드시 필요한 전처리 과정임

> '[UNK]': "Unknown(알 수 없음)" 토큰을 의미한다. 토크나이저가 자신의 사전에 없는 글자를 만나면 이 토큰으로 바꾼다. 한글 '안'이 사전에 없어 [UNK]로 표시된 것으로, 이 토크나이저가 한국어에 최적화되어 있지 않다는 사실을 확인할 수 있다.

> 'ᄀ', '##ᅩ', '##ᄅ', ...: 서브워드(Subword) 토큰화 방식. '고려'라는 단어를 ᄀ, ᅩ, ᄅ, ᅧ 와 같이 더 작은 단위로 쪼갠다. ## 표시는 이 토큰이 바로 앞 토큰과 합쳐져 하나의 단어를 이룬다는 의미이며, 이는 사전에 없는 단어를 처리하기 위한 기술이다.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
tokenizer.tokenize(document.page_content)[:10]

---
**토크나이저의 인코딩 -> 디코딩 과정 확인**

* tokenizer() vs tokenizer.tokenize()
 * **tokenizer.tokenize()**: 텍스트를 토큰 문자열의 리스트로 변환합니다. 사람이 눈으로 토큰 분리 결과를 확인할 때 유용합니다.
 * **tokenizer()**: 텍스트를 모델의 실제 입력값인 숫자 토큰 ID의 리스트(input_ids)가 포함된 딕셔너리로 변환합니다.

 *tokenizer.decode(...): 숫자로 된 토큰 ID 리스트를 입력받아 사람이 읽을 수 있는 문자열로 다시 변환한다. 텍스트를 모델이 사용하는 숫자 ID들로 바꾼 뒤, 그중 앞 11개만 골라서 다시 사람이 읽을 수 있는 글자로 변환

In [ ]:
tokenizer.decode(tokenizer(document.page_content)['input_ids'][:11])

---


In [ ]:
def transformer_len(text):
    return text_splitter.count_tokens(text=text)

text_splitter = SentenceTransformersTokenTextSplitter(tokens_per_chunk=100, chunk_overlap=0, length_function=transformer_len)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], [transformer_len(text) for text in texts], texts[1], \
'|'.join(token for token in tokenizer.tokenize(texts[1]))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

from google.colab import userdata
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
tokenizer.tokenize(document.page_content)[:10]

In [ ]:
text_splitter = SentenceTransformersTokenTextSplitter(tokens_per_chunk=100, chunk_overlap=0, model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', length_function=transformer_len)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], [transformer_len(text) for text in texts], texts[1], \
'|'.join(token for token in tokenizer.tokenize(texts[1]))

7. HuggingFace
- https://python.langchain.com/docs/how_to/split_by_token/#sentencetransformers

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer, GPT2Tokenizer

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('LGAI-EXAONE/EXAONE-4.0-1.2B')
tokenizer.tokenize(texts[1])[:10]

In [ ]:
def huggingface_len(text):
    return len(tokenizer.tokenize(text))

text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(document.page_content)
len(texts), [len(text) for text in texts], [huggingface_len(text) for text in texts], \
'|'.join(token for token in tokenizer.tokenize(texts[1]))